In [ ]:
from scipy.sparse import diags, block_diag, eye
from scipy.sparse.linalg import spsolve, splu

import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

# Simulation of turing pattern in 1D

The systeme of reaction-diffusion is the following one :

$$
\left\{
\begin{array}{l}
\partial_t y_1 =  d_1 \partial^2 _x y_1 + a - (b+1) y_1 + y_1^2y_2 \\
\partial_t y_2 =  d_2 \partial^2 _x y_2 + b y_1 - y_1^2y_2,
\end{array}
\right.
$$

with Neumann boundary conditions.

## Integration with IMEX method

In [ ]:
def imex(tini, tend, nt, yini, a, f, nt_eval=None, param=None):
    
    if param is not None:
          f = lambda x, f=f : f(x, *param)

    dt = (tend-tini) / (nt-1)
    
    lu = splu(eye(yini.size, format="csc") - dt*a)

    if nt_eval is None: nt_eval = (nt,)
        
    y = np.zeros((len(nt_eval), yini.size))
        
    yn = np.copy(yini)

    it_eval = 0 
    for it in range(1,nt):
        yn = lu.solve(yn + dt*f(yn))
        if (it+1 in nt_eval):
            y[it_eval] = yn 
            it_eval = it_eval+1
            
    if nt_eval is None: y[-1] = yn

    return y

In [ ]:
## model parameters
a = 2.
b = 3.
d1 = 1.
d2 = 8.
L = 20.
xmin = 0.
xmax = L

## "amplitude" of the initial perturbation
eps = 0.01

## diffusion part 
nx = 101
dx = (xmax-xmin) / (nx-1)
cst1 = d1/(dx*dx)
cst2 = d2/(dx*dx)

diag0 = np.repeat(-2, nx)
diag1 = np.repeat([2., 1., 2], [1, nx-2, 1])
lap = diags([diag1[1:],  diag0, diag1[:-1]], [-1, 0, 1])
laplap = block_diag((cst1*lap, cst2*lap))

# reaction part
def f(sol, a, b):
    ntot = sol.size
    y1 = sol[:ntot//2]
    y2 = sol[ntot//2:]
    y1y1y2 = y1**2 * y2
    f1 = a - (b+1)*y1 + y1y1y2
    f2 = b*y1 - y1y1y2
    return np.concatenate((f1, f2))

## time integration
tini = 0
tend = 200
nt = 1001
dt = (tend-tini) / (nt-1)

# integration
y1_ini = a + eps*(2*np.random.random(nx)-1)
y2_ini = b/a + eps*(2*np.random.random(nx)-1)
y_ini = np.append(y1_ini, y2_ini)
y = imex(tini, tend, nt, y_ini, laplap, f, param=(a,b), nt_eval=(nt//4, nt//2, (3*nt)//4, nt))
y1 = y[:,:nx]
y2 = y[:,nx:]


# output
x = np.linspace(xmin, xmax, nx)
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=x, y=y1_ini, name='it = 0', line_color='rgb(76,114,176)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=y2_ini, showlegend=False, line_color='rgb(76,114,176)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=y1[0], name='it = nt/4', line_color='rgb(221,132,82)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=y2[0], showlegend=False, line_color='rgb(221,132,82)' ), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=y1[1], name='it = nt/2', line_color='rgb(85,168,104)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=y2[1], showlegend=False, line_color='rgb(85,168,104)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=y1[2], name='it = 3nt/4', line_color='rgb(196,78,82)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=y2[2], showlegend=False, line_color='rgb(196,78,82)'), col=1, row=2)
fig.add_trace(go.Scatter(x=x, y=y1[-1], name='it = nt', line_color='rgb(129,114,179)'), col=1, row=1)
fig.add_trace(go.Scatter(x=x, y=y2[-1], showlegend=False, line_color='rgb(129,114,179)'), col=1, row=2)
fig.update_xaxes(title_text="x", col=1)
fig.update_yaxes(title_text="y1", row=1)
fig.update_yaxes(title_text="y2", row=2)
fig.update_layout(height=1000)
fig.show()